In [1]:
!pip install veroviz
!pip install polyline

In [2]:
import veroviz as vrv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from ipyleaflet import Map, Marker, Polyline
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import json
import requests
import os
import polyline

In [3]:
# Setting up matplotlib to display inline in Jupyter notebooks
%matplotlib inline

In [4]:
# Load drone events from the provided JSON file
with open('drone_events.json', 'r') as file:
    drone_events = json.load(file)

# Load the drone path from the provided JSON file
with open('drone_path.json', 'r') as file:
    drone_path = json.load(file)

# Load vehicle events from the provided JSON file
with open('vehicle_events.json', 'r') as file:
    vehicle_events = json.load(file)

# Load the vehicle path from the provided JSON file
with open('vehicle_path.json', 'r') as file:
    vehicle_path = json.load(file)

In [5]:
drone_events

[{'id': 1,
  'event_type': 'DEPART',
  'stop_id': 'VE',
  'time': 0,
  'coordinates': {'lat': 34.007369, 'lng': -118.143927}},
 {'id': 2,
  'event_type': 'ARRIVE',
  'stop_id': 'TG',
  'time': 1509.7,
  'coordinates': {'lat': 34.088467, 'lng': -118.284521}},
 {'id': 3,
  'event_type': 'DEPART',
  'stop_id': 'TG',
  'time': 1536.1000000000001,
  'coordinates': {'lat': 34.088467, 'lng': -118.284521}},
 {'id': 4,
  'event_type': 'ARRIVE',
  'stop_id': 'GP',
  'time': 1561.5000000000002,
  'coordinates': {'lat': 34.088709, 'lng': -118.284839}},
 {'id': 5,
  'event_type': 'DEPART',
  'stop_id': 'GP',
  'time': 1595.5000000000002,
  'coordinates': {'lat': 34.088709, 'lng': -118.284839}},
 {'id': 6,
  'event_type': 'ARRIVE',
  'stop_id': 'HT',
  'time': 1632.9000000000003,
  'coordinates': {'lat': 34.088717, 'lng': -118.286484}},
 {'id': 7,
  'event_type': 'DEPART',
  'stop_id': 'HT',
  'time': 1705.4000000000003,
  'coordinates': {'lat': 34.088717, 'lng': -118.286484}},
 {'id': 8,
  'event_t

In [6]:
vehicle_events

[{'id': 1,
  'event_type': 'DEPART',
  'stop_id': 'VE',
  'time': 0,
  'coordinates': {'lat': 34.007369, 'lng': -118.143927}},
 {'id': 2,
  'event_type': 'ARRIVE',
  'stop_id': 'TG',
  'time': 1509.7,
  'coordinates': {'lat': 34.088467, 'lng': -118.284521}},
 {'id': 3,
  'event_type': 'SERVICE_START',
  'stop_id': 'TG',
  'time': 1509.7,
  'coordinates': {'lat': 34.088467, 'lng': -118.284521}},
 {'id': 4,
  'event_type': 'SERVICE_END',
  'stop_id': 'TG',
  'time': 1536.1000000000001,
  'coordinates': {'lat': 34.088467, 'lng': -118.284521}},
 {'id': 5,
  'event_type': 'DEPART',
  'stop_id': 'TG',
  'time': 1536.1000000000001,
  'coordinates': {'lat': 34.088467, 'lng': -118.284521}},
 {'id': 6,
  'event_type': 'ARRIVE',
  'stop_id': 'GP',
  'time': 1561.5000000000002,
  'coordinates': {'lat': 34.088709, 'lng': -118.284839}},
 {'id': 7,
  'event_type': 'SERVICE_START',
  'stop_id': 'GP',
  'time': 1561.5000000000002,
  'coordinates': {'lat': 34.088709, 'lng': -118.284839}},
 {'id': 8,
  '

In [7]:
# Extract the first stop's coordinates from the vehicle path to center the map
first_vehicle_stop = vehicle_path[0]['coordinates']
map_center = [first_vehicle_stop['lat'], first_vehicle_stop['lng']]

# Initialize a folium map centered on the first vehicle stop
m = folium.Map(location=map_center, zoom_start=12)

In [8]:
# This function is necessary to ensure that the path plotted for vehicles adheres to actual roadways,
# providing a more realistic visualization of the route.
def get_osrm_route(start, end, server='http://router.project-osrm.org'):
    """
    Queries the Open Source Routing Machine (OSRM) for the best route between two geographic points.

    Parameters:
    - start: A tuple containing the latitude and longitude of the starting point (lat, lng).
    - end: A tuple containing the latitude and longitude of the ending point (lat, lng).
    - server: The URL of the OSRM server to query. Defaults to the public OSRM demo server.

    Returns:
    - A list of (latitude, longitude) tuples representing the decoded polyline of the route.
    """

    # Format the coordinates for the OSRM query. OSRM expects coordinates in the order of longitude,latitude.
    coords = f"{start[1]},{start[0]};{end[1]},{end[0]}"  # Note the reversal from (lat, lng) to (lng, lat).

    # Construct the URL for the OSRM route service, specifying 'overview=full' to get the full route geometry.
    url = f"{server}/route/v1/driving/{coords}?overview=full"

    # Send a GET request to the OSRM server and parse the JSON response.
    response = requests.get(url).json()

    # Extract the encoded polyline that represents the route's geometry.
    encoded_polyline = response['routes'][0]['geometry']

    decoded_polyline = polyline.decode(encoded_polyline)
    return decoded_polyline

# Plot vehicle path
for i in range(len(vehicle_path)-1):
    start = (vehicle_path[i]['coordinates']['lat'], vehicle_path[i]['coordinates']['lng'])
    end = (vehicle_path[i+1]['coordinates']['lat'], vehicle_path[i+1]['coordinates']['lng'])
    # Get route from OSRM
    corrected_route = get_osrm_route(start, end)
    # Add polyline to the map
    folium.PolyLine(locations=corrected_route, color='blue', weight=2.5).add_to(m)

launch_land_indices = []
for i, event in enumerate(drone_events):
    if event['event_type'] == 'LAUNCH' or event['event_type'] == 'LAND':
        launch_land_indices.append(i)

# Assuming your initial drawing logic here for all paths, before differentiating launch/land paths

# Now draw the distinct launch/land segments
for i in range(len(launch_land_indices) - 1):
    start_index = launch_land_indices[i]
    end_index = launch_land_indices[i + 1]

    # Check if the segment should be drawn as a drone flight path
    if drone_events[start_index]['event_type'] == 'LAUNCH':
        # Extract coordinates for the flight path
        flight_path = [[drone_events[j]['coordinates']['lat'], drone_events[j]['coordinates']['lng']]
                       for j in range(start_index, end_index + 1)]

        # Draw the flight path
        folium.PolyLine(
            locations=flight_path,
            color='green',
            weight=2.5,
            dash_array='5'  # This makes the line dashed
        ).add_to(m)

# Add markers for each vehicle event on the map
for event in vehicle_events:
    folium.Marker(
        location=[event['coordinates']['lat'], event['coordinates']['lng']],
        popup=f"Vehicle Event: {event['event_type']}",
        # Use blue icons for vehicle events for consistency with the path color
        icon=folium.Icon(color='blue')
    ).add_to(m)

# Add markers for each drone event on the map
for event in drone_events:
    folium.Marker(
        location=[event['coordinates']['lat'], event['coordinates']['lng']],
        popup=f"Drone Event: {event['event_type']}",
        # Use green icons for drone events to match the path color
        icon=folium.Icon(color='green')
    ).add_to(m)

In [9]:
m

In [10]:
nodes_df = pd.DataFrame({
    # Extracting node IDs and constructing basic node information
    'id': [item['id'] for item in vehicle_path],
    'lat': [item['coordinates']['lat'] for item in vehicle_path],
    'lon': [item['coordinates']['lng'] for item in vehicle_path],
    'altMeters': 0,
    # Node names and types for identification and classification
    'nodeName': ['Node ' + str(item['id']) for item in vehicle_path],
    'nodeType': 'Waypoint',
    'popupText': ['Node ' + str(item['id']) for item in vehicle_path],
    # Leaflet options for map visualization
    'leafletIconPrefix': 'fa',
    'leafletIconType': 'flag',
    'leafletColor': 'blue',
    'leafletIconText': [str(item['id']) for item in vehicle_path],
    # Cesium options for 3D globe visualization
    'cesiumIconType': 'pin',
    'cesiumColor': 'red',
    'cesiumIconText': [str(item['id']) for item in vehicle_path],
    'elevMeters': [0 for _ in vehicle_path]
})

nodes_df

,id,lat,lon,altMeters,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,elevMeters
0,1,34.007369,-118.143927,0,Node 1,Waypoint,Node 1,fa,flag,blue,1,pin,red,1,0
1,2,34.088467,-118.284521,0,Node 2,Waypoint,Node 2,fa,flag,blue,2,pin,red,2,0
2,3,34.088709,-118.284839,0,Node 3,Waypoint,Node 3,fa,flag,blue,3,pin,red,3,0
3,4,34.088717,-118.286484,0,Node 4,Waypoint,Node 4,fa,flag,blue,4,pin,red,4,0
4,5,34.089727,-118.285530,0,Node 5,Waypoint,Node 5,fa,flag,blue,5,pin,red,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,103,34.094216,-118.293212,0,Node 103,Waypoint,Node 103,fa,flag,blue,103,pin,red,103,0
103,104,34.091351,-118.293884,0,Node 104,Waypoint,Node 104,fa,flag,blue,104,pin,red,104,0
104,105,34.090390,-118.293879,0,Node 105,Waypoint,Node 105,fa,flag,blue,105,pin,red,105,0
105,106,34.096583,-118.294127,0,Node 106,Waypoint,Node 106,fa,flag,blue,106,pin,red,106,0


In [11]:
 #Initialize a new DataFrame for storing nodes using VeRoViz's initDataframe function
newNodes = vrv.initDataframe('Nodes')
# Initialize a new DataFrame for storing arcs using VeRoViz's initDataframe function
newArcs = vrv.initDataframe('Arcs')
#newArcs
newNodes

,id,lat,lon,altMeters,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,elevMeters


In [12]:
def build_paths(events):
    # Initialize storage for paths and the working current path
    paths = []
    current_path = []
    last_event_type = ""

    # Sort events by their timestamp to process in chronological order
    sorted_events = sorted(events, key=lambda x: x['time'])

    # Loop through sorted events to construct paths based on event types
    for event in sorted_events:
        # Start or restart path if the current event is a departure
        if event['event_type'] == 'DEPART':
            # Append the current path to paths if ending conditions are met
            if last_event_type in ['ARRIVE', 'SERVICE_START'] and current_path:
                paths.append(current_path)
                current_path = []
            # Add current event to the path
            current_path.append(event)
        # Add event to current path if it is an arrival and update state
        elif event['event_type'] == 'ARRIVE':
            current_path.append(event)
            last_event_type = 'ARRIVE'
        # Update state if the event is a service start
        elif event['event_type'] == 'SERVICE_START':
            last_event_type = 'SERVICE_START'

    # Add any remaining events in the current path to paths
    if current_path:
        paths.append(current_path)

    return paths

In [13]:
# Dictionary to store 3D model files and color properties for different vehicle types
vehicleProperties = {
    'drone': {
        'modelFile': 'veroviz/models/drone.gltf',  # Path to the 3D model file for drones
        'color': 'red'                             # Display color for drone models in visualizations
    },
    'truck': {
        'modelFile': 'veroviz/models/ub_truck.gltf',  # Path to the 3D model file for trucks
        'color': 'blue'                               # Display color for truck models in visualizations
    }
}

In [14]:
# Dictionary to hold the paths for drone and truck events processed by the build_paths function
mySolution = {
    'drone': build_paths(drone_events),
    'truck': build_paths(vehicle_events)
}

drone_paths = mySolution['drone']

truck_paths = mySolution['truck']


In [15]:
# Define the structure of the DataFrame with specific columns for route assignments
columns = [
    'odID', 'objectID', 'modelFile', 'startLat', 'startLon', 'endLat', 'endLon',
    'startTimeSec', 'endTimeSec', 'leafletColor', 'leafletWeight', 'leafletStyle',
    'leafletOpacity', 'leafletCurveType', 'leafletCurvature', 'arcCurveType', 'useArrows',
    'cesiumColor', 'cesiumWeight', 'cesiumStyle', 'cesiumOpacity', 'popupText',
    'startElevMeters', 'endElevMeters', 'startAltMeters', 'endAltMeters'
]
# Initialize an empty DataFrame for holding assignment data with predefined columns
assignments = pd.DataFrame(columns=columns)

# Starting unique identifier for each route segment
odID = 1

# Iterate over each path in the list of drone paths
for path in drone_paths:
    # Loop through each event in the path to create segments
    for i in range(len(path) - 1):
        # Start and end events for the current segment
        start_event = path[i]
        end_event = path[i + 1]

        # Create a dictionary for the current row of the DataFrame
        row = {
            'odID': odID,  # Unique identifier for the segment
            'objectID': 'drone',  # Type of vehicle
            'modelFile': vehicleProperties['drone']['modelFile'],  # 3D model file path for the drone
            'startLat': start_event['coordinates']['lat'],  # Starting latitude
            'startLon': start_event['coordinates']['lng'],  # Starting longitude
            'endLat': end_event['coordinates']['lat'],  # Ending latitude
            'endLon': end_event['coordinates']['lng'],  # Ending longitude
            'startTimeSec': start_event['time'],  # Start time in seconds
            'endTimeSec': end_event['time'],  # End time in seconds
            'leafletColor': vehicleProperties['drone']['color'],  # Color for Leaflet map visualization
            'leafletWeight': 5,  # Line weight for Leaflet
            'leafletStyle': 'solid',  # Line style for Leaflet
            'leafletOpacity': 0.8,  # Line opacity for Leaflet
            'leafletCurveType': 'straight',  # Type of curve for Leaflet visualization
            'leafletCurvature': 0.2,  # Curvature parameter for Leaflet, if applicable
            'arcCurveType': 'straight',  # Type of curve for other visualizations
            'useArrows': True,  # Whether to use arrows for direction indication
            'cesiumColor': vehicleProperties['drone']['color'],  # Color for Cesium 3D visualization
            'cesiumWeight': 5,  # Line weight for Cesium
            'cesiumStyle': 'solid',  # Line style for Cesium
            'cesiumOpacity': 0.8,  # Line opacity for Cesium
            'popupText': f'From {start_event["stop_id"]} to {end_event["stop_id"]}',  # Popup text for the segment
            'startElevMeters': 0,  # Starting elevation, defaulting to 0
            'endElevMeters': 0,  # Ending elevation, defaulting to 0
            'startAltMeters': 10,  # Constant altitue
            'endAltMeters': 10
        }

        # Increment the unique identifier for the next route segment
        odID += 1

        # Convert the dictionary to a pandas Series (one row of DataFrame) with the same columns order
        row = pd.Series(row, index=assignments.columns)

        # Append the row to the DataFrame, ignoring index to avoid duplication
        assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_index=True)

/var/folders/f3/y0qnsppj4l97jv5nrbhfry2h0000gn/T/ipykernel_81841/3608213011.py:60: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_index=True)
/var/folders/f3/y0qnsppj4l97jv5nrbhfry2h0000gn/T/ipykernel_81841/3608213011.py:60: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_index=True)
/var/folders/f3/y0qnsppj4l97jv5nrbhfry2h0000gn/T/ipykernel_81841/3608213011.py:60: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_i

In [16]:
# Create an interactive Leaflet map using data from nodes and arcs DataFrames
# 'nodes_df' contains geographical and visual data about nodes (locations)
# 'assignments' contains data about paths or routes connecting these nodes
map_object = vrv.createLeaflet(
    nodes=nodes_df,
    arcs=assignments
)

map_object

/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_onl

In [17]:
# Define the structure of the DataFrame with specific columns for route assignments
columns = [
    'odID', 'objectID', 'modelFile', 'startLat', 'startLon', 'endLat', 'endLon',
    'startTimeSec', 'endTimeSec', 'leafletColor', 'leafletWeight', 'leafletStyle',
    'leafletOpacity', 'leafletCurveType', 'leafletCurvature', 'arcCurveType', 'useArrows',
    'cesiumColor', 'cesiumWeight', 'cesiumStyle', 'cesiumOpacity', 'popupText',
    'startElevMeters', 'endElevMeters', 'startAltMeters', 'endAltMeters'
]
# Initialize an empty DataFrame for holding assignment data with predefined columns
assignments = pd.DataFrame(columns=columns)

# Starting unique identifier for each route segment
odID = 1


# Iterate over each path in the list of vehicle paths
for path in truck_paths:  # Assuming 'vehicle_paths' is your dataset for vehicle routes
    # Loop through each event in the path to create segments
    for i in range(len(path) - 1):
        # Start and end events for the current segment
        start_event = path[i]
        end_event = path[i + 1]

        # Create a dictionary for the current row of the DataFrame
        row = {
            'odID': odID,  # Unique identifier for the segment
            'objectID': 'truck',  # Type of vehicle
            'modelFile': vehicleProperties['truck']['modelFile'],  # 3D model file path for the vehicle
            'startLat': start_event['coordinates']['lat'],  # Starting latitude
            'startLon': start_event['coordinates']['lng'],  # Starting longitude
            'endLat': end_event['coordinates']['lat'],  # Ending latitude
            'endLon': end_event['coordinates']['lng'],  # Ending longitude
            'startTimeSec': start_event['time'],  # Start time in seconds
            'endTimeSec': end_event['time'],  # End time in seconds
            'leafletColor': vehicleProperties['truck']['color'],  # Color for Leaflet map visualization
            'leafletWeight': 5,  # Line weight for Leaflet
            'leafletStyle': 'solid',  # Line style for Leaflet
            'leafletOpacity': 0.8,  # Line opacity for Leaflet
            'leafletCurveType': 'straight',  # Type of curve for Leaflet visualization
            'leafletCurvature': 0,  # Curvature parameter for Leaflet, set to 0 for straight routes
            'arcCurveType': 'straight',  # Type of curve for other visualizations, straight for ground vehicles
            'useArrows': True,  # Whether to use arrows for direction indication
            'cesiumColor': vehicleProperties['truck']['color'],  # Color for Cesium 3D visualization
            'cesiumWeight': 5,  # Line weight for Cesium
            'cesiumStyle': 'solid',  # Line style for Cesium
            'cesiumOpacity': 0.8,  # Line opacity for Cesium
            'popupText': f'From {start_event["stop_id"]} to {end_event["stop_id"]}', # Popup text for the segment
            'startAltMeters': 0,  # Constant altitue
            'endAltMeters': 0
            }

        # Increment the unique identifier for the next route segment
        odID += 1

        # Convert the dictionary to a pandas Series with the correct columns order
        row = pd.Series(row, index=assignments.columns)

        # Append the row to the DataFrame, ignoring index to avoid duplication
        assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_index=True)

# Output the completed assignments DataFrame
print(assignments)

    odID objectID                     modelFile   startLat    startLon  \
0      1    truck  veroviz/models/ub_truck.gltf  34.007369 -118.143927   
1      2    truck  veroviz/models/ub_truck.gltf  34.088467 -118.284521   
2      3    truck  veroviz/models/ub_truck.gltf  34.088709 -118.284839   
3      4    truck  veroviz/models/ub_truck.gltf  34.088717 -118.286484   
4      5    truck  veroviz/models/ub_truck.gltf  34.089727 -118.285530   
..   ...      ...                           ...        ...         ...   
101  102    truck  veroviz/models/ub_truck.gltf  34.093294 -118.292799   
102  103    truck  veroviz/models/ub_truck.gltf  34.094216 -118.293212   
103  104    truck  veroviz/models/ub_truck.gltf  34.091351 -118.293884   
104  105    truck  veroviz/models/ub_truck.gltf  34.090390 -118.293879   
105  106    truck  veroviz/models/ub_truck.gltf  34.096583 -118.294127   

        endLat      endLon startTimeSec  endTimeSec leafletColor  ...  \
0    34.088467 -118.284521            

/var/folders/f3/y0qnsppj4l97jv5nrbhfry2h0000gn/T/ipykernel_81841/2219359892.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_index=True)
/var/folders/f3/y0qnsppj4l97jv5nrbhfry2h0000gn/T/ipykernel_81841/2219359892.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_index=True)
/var/folders/f3/y0qnsppj4l97jv5nrbhfry2h0000gn/T/ipykernel_81841/2219359892.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  assignments = pd.concat([assignments, pd.DataFrame([row])], ignore_i

In [18]:
truck_map_object = vrv.createLeaflet(
    nodes=nodes_df,
    arcs=assignments
)

truck_map_object

/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_onl

In [19]:
assignments['modelScale'] = 100
assignments['modelMinPxSize'] = 75
assignments['ganttColor'] = 'darkgray'
assignments['wayname'] = None
assignments['waycategory'] = None
assignments['surface'] = None
assignments['waytype'] = None
assignments['steepness'] = None
assignments['tollway'] = None

In [22]:
vrv.createCesium(assignments=assignments,
                 nodes=nodes_df,
                 cesiumDir=os.getcwd()+'/Cesium-1.116',
                 problemDir='veroviz/vrpd')

/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_onl

Message: File selector was written to /Users/max/vrpd/Cesium-1.116/veroviz/vrpd/;veroviz;vrpd.vrv ...
Message: Configs were written to /Users/max/vrpd/Cesium-1.116/veroviz/vrpd/config.js ...
Message: Nodes were written to /Users/max/vrpd/Cesium-1.116/veroviz/vrpd/displayNodes.js ...
Message: Assignments (.js) were written to /Users/max/vrpd/Cesium-1.116/veroviz/vrpd/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/max/vrpd/Cesium-1.116/veroviz/vrpd/routes.czml ...


/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  tmpSubAssignment = pd.concat([tmpSubAssignment, collection.loc[i: i, :].copy()], ignore_index= True, sort=True)
/Users/max/miniforge3/envs/vrpd/lib/python3.12/site-packages/veroviz/_deconstructAssignments.py:59: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_onl